## Importing modules and packages

In [1]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import os.path

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching
import gr_utils # Contains useful geometry functions

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Input parameters

In [2]:
trailname = 'gr16' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)
delta = 0.005 # Tolerance around bounding box per trail section [deg]
points_per_batch = 100 # Subdivide the trail into batches of this many points

## Loading GPX file

In [3]:
filename_gpx = 'data_input/' + trailname + '.gpx'
filename_csv = 'data_output/' + trailname + '.csv'
if not os.path.isfile(filename_csv): # The GPX file was not processed into a clean CSV file before
    if not os.path.isfile(filename_gpx): # The GPX file does not exist, throw error
        raise ValueError(f'The GPX file <{filename_gpx}> was not found! Please make sure it exists.')
    else: # The GPX file exists, so convert it into a clean CSV file
        print(f'Converting GPX file <{filename_gpx}> into cleaned CSV file <{filename_csv}>')
        gr_utils.process_gpx(filename_gpx,filename_csv)
        print('Completed conversion!')
print(f'Loading trail points from <{filename_gpx}>')
trail = pd.read_csv(filename_csv) # Now read the cleaned CSV file into a DataFrame (latitude, longitude, elevation)
print('Loaded!')

Loading trail points from <data_input/gr16.gpx>
Loaded!


## Processing sections of trail

In [5]:
n_trail = len(trail) # Number of GPX points in the trail
n_batch = int(np.ceil(trail.shape[0]/points_per_batch)) # Number of batches to be run
# for b in range(n_batch): # Using batch counter b
for b in range(1):
    
    # Define the range of GPX points to process in the current batch
    n1 = b*points_per_batch # First point of this batch
    n2 = min(n1 + points_per_batch, n_trail) # Last point of this batch (clipped)
    trail_section = trail.loc[n1:n2] # Select that range of GPX points
    trail_coords  = gr_mapmatch.trail_to_coords(trail_section) # Convert the points into a list of [lat, lon] pairs
    
    # Check if this batch was processed before
    batch_out = f'cache/{trailname}_{n1}to{n2}.csv'
    print(f'Processing batch {b} of {n_batch-1}, covering GPX track points {n1} through {n2}')
    if os.path.isfile(batch_out): # It already exists
        print('   This batch was processed before, skipping.')
    else: # It does not exist, so process it
        print('   Processing batch...')
        network, segment_list = gr_mapmatch.match_batch(trail_section, trail_coords, delta)
        gr_mapmatch.write_batch(batch_out, segment_list)

Processing batch 0 of 60, covering GPX track points 0 through 100
   Processing batch...


/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


   Processing street network...
Beginning main processing loop...
Handling GPX point 100 of 100
Handling node_list pair 50 of 50
Handling route_list pair 65 of 65
Writing outputs to file...
